In [1]:
#importing requirments
import pandas as pd
import ollama
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

creating embeddings

In [2]:

def create_embeddings(l):
    res = ollama.embed(
        model="qllama/bge-large-en-v1.5:q4_k_m",
        input=l
    )

    vector = res["embeddings"]
    return vector



In [3]:
# s = "my name is keyur"
# print(create_embeddings(s))

In [4]:
# l=["hi","by"]
# print(len(create_embeddings(l)))


In [ ]:

jsons = os.listdir("jsons")
my_dict = []
chunk_id =0
for json_file in jsons:
    with open(f"jsons/{json_file}") as f:
        content = json.load(f)
    print(f"creating embeddings for {json_file}")
    embeddings =create_embeddings([c['text'] for c in content["chunks"]])
    for i,chunk  in enumerate(content["chunks"]):
        chunk["chunk_id"] = chunk_id
        chunk["embeddings"] = embeddings[i]
        chunk_id += 1
        my_dict.append(chunk)
        
    
# print(my_dict)

creating embeddings for 01_Introduction to Programming .json
creating embeddings for 02_Some Amazing Python Programs .json
creating embeddings for 03_Modules and Pip .json
creating embeddings for 04_Our First Python Program .json
creating embeddings for 05_Comments.json


saving model

In [ ]:

df = pd.DataFrame.from_records(my_dict)
df.to_pickle("data.pkl")

NameError: name 'my_dict' is not defined

pulling top matching chunks for incoming query and introspecting it

In [4]:
df = pd.read_pickle("data.pkl")
# df = pd.read_csv("embed_data.csv")    #we are not using csv cause it stored embeddings as list hence we didnt want that so were using pkl to store

In [5]:
df["embeddings"].dtype

dtype('O')

In [6]:
print(np.vstack(df['embeddings']).shape)

(1678, 1024)


In [6]:
incoming_query = input("enter a query :")
question_embedding = create_embeddings(incoming_query)[0]
# print(np.vstack(df['embeddings']).shape)
similarities  =cosine_similarity(np.vstack(df['embeddings']),[question_embedding]).flatten()
top_results = 5
max_idx = similarities.argsort()[::-1][0:top_results]

In [8]:
new_df  = df.loc[max_idx]
print(new_df[["number","title","text"]])    

     number                         title  \
1501     05                      Comments   
804      03              Modules and Pip    
61       01  Introduction to Programming    
1582     05                      Comments   
1545     05                      Comments   

                                               text  
1501                            tun tun tun tun tun  
804                                              hi  
61                                           Raplet  
1582                     from tilde character, like  
1545   followed by the character you want to insert  


In [21]:
context_blocks = []
for index,item in new_df.iterrows():
    context_blocks.append(f'''lecure title:{item["title"]},lecure number:{item["number"]},lecture text:{item["text"]},lecture timings:{item["Start"]}-{item["end"]}''')
print(context_blocks)

['lecure title:Comments,lecure number:05,lecture text: tun tun tun tun tun,lecture timings:754.0-756.0', 'lecure title:Modules and Pip ,lecure number:03,lecture text: hi,lecture timings:454.0-456.0', 'lecure title:Introduction to Programming ,lecure number:01,lecture text: Raplet,lecture timings:124.0-126.0', 'lecure title:Comments,lecure number:05,lecture text: from tilde character, like,lecture timings:916.0-918.0', 'lecure title:Comments,lecure number:05,lecture text: followed by the character you want to insert,lecture timings:842.0-844.0']


In [26]:
prompt = f'''You are a teaching assistant.

Use ONLY the lecture content below.
Follow this STRICT format:

Topic:
<one line>

Lecture details:
- Title:
- Lecture number:
- Video timestamps:

Explanation:
<short explanation>

Learning steps:
- Step 1
- Step 2

Do NOT combine title and lecture number.
Do NOT invent names.
Do NOT use general programming knowledge.
Use ONLY the lecture content provided, even if it seems incomplete.


Lecture content:
{context_blocks}

Question:
{incoming_query}


'''
with open("promp.txt","w") as f:
    f.write(prompt)

# getting response from llm
now were using llama2.3:3b which is small in size and faster


In [27]:
def inference(prompt):
    model =ollama.generate(
        model="llama3.2:3b",
        prompt=prompt,
        stream=False
    )
    response =  model["response"]
    return response


In [28]:
response = inference(prompt=prompt)

In [29]:
with open("output.txt","w") as f:
    f.write(response)

# Now this section is for merged chunks 

In [5]:

jsons = os.listdir("merge_json")
my_dict = []
chunk_id =0
for json_file in jsons:
    with open(f"merge_json/{json_file}") as f:
        content = json.load(f)
    print(f"creating embeddings for {json_file}")
    embeddings =create_embeddings([c['text'] for c in content["chunks"]])
    for i,chunk  in enumerate(content["chunks"]):
        chunk["chunk_id"] = chunk_id
        chunk["embeddings"] = embeddings[i]
        chunk_id += 1
        my_dict.append(chunk)
        
    
# print(my_dict)

creating embeddings for 01_Introduction to Programming .json
creating embeddings for 02_Some Amazing Python Programs .json
creating embeddings for 03_Modules and Pip .json
creating embeddings for 04_Our First Python Program .json
creating embeddings for 05_Comments.json


In [6]:

df = pd.DataFrame.from_records(my_dict)
df.to_pickle("merge_data.pkl")

In [7]:
df = pd.read_pickle("merge_data.pkl")

In [8]:
incoming_query = input("enter a query :")
question_embedding = create_embeddings(incoming_query)[0]
# print(np.vstack(df['embeddings']).shape)
similarities  =cosine_similarity(np.vstack(df['embeddings']),[question_embedding]).flatten()
top_results = 5
max_idx = similarities.argsort()[::-1][0:top_results]

In [9]:
new_df  = df.loc[max_idx]
print(new_df[["number","title","text"]])    

    number                         title  \
185     03              Modules and Pip    
336     05                      Comments   
18      01  Introduction to Programming    
66      01  Introduction to Programming    
129     03              Modules and Pip    

                                                  text  
185   in 24 hours  so this video was  trending  so ...  
336   if you have not accessed this code yet  then ...  
18    Rapple  So here you can see  That my video  Y...  
66    Like the video and subscribe the channel  So ...  
129   and you will be able to see this video  and a...  


In [ ]:
prompt = f'''You are a teaching assistant.

Use ONLY the lecture content below.
Follow this STRICT format:

Topic:
<one line>

Lecture details:
- Title:
- Lecture number:
- Video timestamps:

Explanation:
<short explanation>

Learning steps:
- Step 1
- Step 2

Do NOT combine title and lecture number.
Do NOT invent names.
Do NOT use general programming knowledge.
Use ONLY the lecture content provided, even if it seems incomplete.


Lecture content:
{context_blocks}

Question:
{incoming_query}

'''
with open("merge_promp.txt","w") as f:
    f.write(prompt)

In [27]:
def inference(prompt):
    model =ollama.generate(
        model="llama3.2:3b",
        prompt=prompt,
        stream=False
    )
    response =  model["response"]
    return response


In [28]:
response = inference(prompt=prompt)

In [29]:
with open("merge_output.txt","w") as f:
    f.write(response)